# Einführung Multi-Variate Datenanalyse (MVDA)

## Teil 1: Visualisierung hoch-dimensionaler Datensätze mit Python
für Kurse Mathematische Berechnungssoftware (MBS)

_Simeon Sauer, SS2020_

inspired by https://python-for-multivariate-analysis.readthedocs.io/index.html

## Go where the data is

* Datenmengen explodieren in alle Bereichen unseres Lebens
* Neue Erkenntnisse / Anwendungen / Produkte liegen da, wo (viele) Daten vorhanden sind
* Wenn ich für meine Fragestellung viele Daten habe, was kann ich damit anfangen?
* Immer die erste (und oft die beste) Analyse: Gute Visualierung!

**Dieses Notebook**: Welchen Arten gibt es, die Daten zu visualisieren?

## Python essentials

#### Hello World

In [ ]:
# Das ist ein Kommentar
print("Hallo Welt")  # Funktion 'print' gibt Text aus. Funktionsargument(e) in runden Klammern, Text in Hochkomma.

meine_erste_Variable = "Ich bin ein Text (a.k.a String)"
print(meine_erste_Variable)

meine_zweite_Variable = 4
print(meine_zweite_Variable, "plus 2 ist gleich", meine_zweite_Variable + 2)

#### Bibliotheken laden

In [ ]:
import pandas as pd  # 'import' lädt eine Bibliothek mit weiteren Funktionen. 'as' legt eine Abkürzung fest

help(pd)

import plotly.express as px

## Multivariate Datensätze
multivariat = viele gemessenen Variablen = "hochdimensional"

#### Daten importieren

In [ ]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data", header=None)

data

Was bedeuten die Spalten und Zeilen?
--> http://archive.ics.uci.edu/ml/datasets/Wine
- Zeilen: Unterschiedliche Proben von Weinen aus einer bestimmten Region in Italien
- Spalten: 0 - Rebensorte; 1-13: verschiedene Messwerte

In [ ]:
# Spaltenname einführen:
data.columns = [
    "Cultivar",  # Weinsorte
    "Alcohol",
    "Malic acid",
    "Ash",
    "Alcalinity of ash",
    "Magnesium",
    "Total phenols",
    "Flavanoids",
    "Nonflavanoid phenols",
    "Proanthocyanins",
    "Color intensity",
    "Hue", # Farbton
    "OD280/OD315 of diluted wines", # Verhältnis Optische Dichten bei 280nm / 315 nm Wellenlänge 
    "Proline"]

# Index-Spalte wird separat behandelt in pandas-Bibliothek
data.index.name='sample no.'

# Jetzt nennen wir noch die Weinsorten ('Cultivar') um
Weinsorten = ['Montepulciano', 'Trebbiano d\'Abruzzo', 'Nero d\'Avola']
data['Cultivar'] = data['Cultivar'].replace([1,2,3], Weinsorten)

# Wie sieht die Datentabelle jetzt aus?
data

#### slicing

In [ ]:
# wählen alle Zeilen aus, in denen Weinsorte=Montepulciano und Magenium>100
slice = (data['Cultivar']=='Montepulciano') & (data['Magnesium'] > 100)
print(slice)
data[slice]

#### etwas deskriptive Statistik

In [ ]:
data.describe().round(2)

In [ ]:
data.groupby('Cultivar').mean()

## Explorative Datenanalyse (EDA)

In [ ]:
var_list = list(data.columns[1:8]) # Liste mit Variablen für weitere Plots
for var in var_list:
    fig = px.histogram(data,
                       x=var,
                       color="Cultivar", # einzelne Histogramme für jede Weinsorte
                       nbins=20, # Anzahl Balken ('bins')
                       barmode='group', # gruppiere die Balken für die 3 Weinsorten, nicht überlagern
                       marginal='box', # Boxplot über das Histogram
                       histnorm='percent') # normieren Histogramm auf 100%, nicht auf die Anzahl der Beobachtung
    fig.show() # zeige plot an (nötig wegen 'for' Schleife)

#### scatter matrix plot

In [ ]:
#h = pd.plotting.scatter_matrix(data, figsize=(15,15), hist_kwds={'bins':20})
px.scatter_matrix(data,
                  color='Cultivar',
                  dimensions=var_list,
                  opacity=0.8, # macht Datenpunkte zu 20% durchsichtig
                  height=1000, # Höhe des Gesamtplots in Pixeln
                  hover_name=data.index) # Überschrift der eingeblendeten Mouse-Over-Information: Zeilennr.

#### 3D scatter plots

In [ ]:
px.scatter_3d(data, x='Alcohol', y='Flavanoids', z='Malic acid', color='Cultivar', opacity=0.5, hover_name=data.index)
# px.scatter_3d(data, x='Flavanoids', y='Malic acid', z='Ash', color='Alcohol', symbol='Cultivar', hover_name=data.index)
# px.scatter_3d(data, x='Flavanoids', y='Malic acid', z='Ash', color='Alcohol', symbol='Cultivar', size='Total phenols', hover_name=data.index)

In [ ]:
px.scatter(data, x="Alcohol", y="Flavanoids",
    size="Malic acid", color="Cultivar",
    hover_name=data.index, size_max=30)

#### parallel coordinates

In [ ]:
px.parallel_coordinates(data, dimensions=var_list)

#### spider diagram / radar chart

In [ ]:
fig = go.Figure()
colors = iter(px.colors.qualitative.Plotly)

for grape, subset in data.groupby('Cultivar'):
    subset_normiert = (subset[var_list] - data[var_list].mean())/data[var_list].std()
    newfig = px.line_polar(r=subset_normiert[var_list].mean(),
                           theta=var_list,
                           line_close=True,
                           hover_name=pd.Series([grape]).repeat(len(var_list)))
    trace = newfig.data[0]
    trace.name=grape
    trace.line ={'color': next(colors)}
    fig.add_trace(newfig.data[0])
    fig.update_layout(showlegend=True)
fig.show()

## Fazit
* anhand einzelner Variablen (univariat) lassen sich Weinsorten nicht eindeutig unterscheiden.
* Wenn man Alcohol und Flavanoide zusammen betrachtet, lassen sie sich relativ gut unterscheiden.
* Visualierung nimmt mit Anzahl Variablen (Dimensionen) zu: Sind wir sicher, das wir nichts übersehen haben? Vielleicht gibt es noch bessere Klassifizierungsmethode?